# Use Case: Chatbot for UNEJ Student Service
SIUBOT (Simple UNEJ BOT) is a chatbot designed to assist the academic community and the public in accessing information available on the official website of the University of Jember. This chatbot gathers data through web scraping from the university's official website and is built using the Llama 3.3 70B Versatile model from Groq.

With a Gradio-based visualization, SIUBOT serves as a simple prototype of a university chatbot, enhancing accessibility and efficiency in obtaining official university information.

*Created by Renata Sayidatul*

In [1]:
!pip install firecrawl-py -q

In [2]:
!pip install gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [3]:
!pip install groq -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 4.2 MB/s eta 0:00:00


In [4]:
import getpass
import gradio as gr
from groq import Groq
from firecrawl import FirecrawlApp
from IPython.core.display import Markdown

## Web Scraping

In [5]:
FIRECRAWL_API_KEY = getpass.getpass("Firecrawl API Key: ")
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

Firecrawl API Key: ··········


In [6]:
unej = app.scrape_url(
    'https://unej.ac.id/',
    params={'formats': ['markdown', 'html']}
)
unej_markdown = unej["markdown"]

In [7]:
context = f"""The following are unej_markdown \n\n<unej_markdown>{unej_markdown}</unej_markdown>\n"""

## Setup LLM

In [8]:
GROQ_API_KEY=getpass.getpass("Groq API Key: ")
client = Groq(api_key=GROQ_API_KEY)

Groq API Key: ··········


In [9]:
system_message = f"""
Halo! Kamu adalah asisten virtual yang membantu memberikan informasi terkait Universitas Jember kepada mahasiswa dan masyarakat.

Saat menjawab pertanyaan:
1. Mulailah dengan sapaan yang ramah dan hangat.
2. Jawablah hanya berdasarkan informasi yang tersedia di website resmi Universitas Jember.
3. Jangan menambahkan informasi lain yang tidak bersumber dari website resmi.
4. Jika tidak mengetahui jawabannya atau pertanyaan kurang jelas, jawab dengan:
   "Maaf, saya kurang mengetahui hal tersebut. Untuk informasi lebih lanjut, silakan menghubungi Kemahasiswaan Universitas Jember atau Kemahasiswaan masing-masing Fakultas."
5. Gunakan bahasa yang sesuai dengan bahasa pertanyaan yang diajukan.
6. Akhiri setiap jawaban dengan ucapan terima kasih.
"""

In [10]:
def get_response(query: str, context: str):
  completion = client.chat.completions.create(
      model="llama-3.3-70b-versatile",
      messages=[
          {
              "role": "system",
              "content": system_message
          },
          {
              "role": "user",
              "content": context
          },
          {
              "role": "user",
              "content": query
          }
      ],
      temperature=0,
      max_tokens=2400,
      top_p=1,
      stream=False,
      stop=None,
  )
  return completion.choices[0].message.content

## Manual Testing

In [12]:
answer = get_response(
    query="Apa itu Universitas Jember?",
    context=context,
)
display(Markdown(answer))

Halo! Universitas Jember (UNEJ) adalah Perguruan Tinggi Negeri (PTN) yang berdiri semenjak tahun 1964 di Jember, Jawa Timur. UNEJ merupakan salah satu perguruan tinggi terkemuka di Indonesia yang mengembangkan berbagai rumpun keilmuan, termasuk ilmu pertanian, keteknikan, kesehatan, dan sosial humaniora. Universitas ini memiliki visi dan misi untuk mengembangkan dan memajukan pendidikan, penelitian, dan pengabdian kepada masyarakat. Terima kasih!

In [15]:
answer = get_response(
    query="I want to apply at Faculty of Computer Science in Information System program, could you give me some information about that? And what sholud I do next?",
    context=context,
    )
display(Markdown(answer))

Halo! Saya senang membantu Anda. Universitas Jember memiliki Fakultas Ilmu Komputer, bukan Fakultas Ilmu Komputer saja, tetapi juga memiliki program studi Sistem Informasi di bawah naungan Fakultas Ilmu Komputer.

Berikut adalah beberapa informasi tentang program studi Sistem Informasi di Fakultas Ilmu Komputer Universitas Jember:

* Program studi Sistem Informasi di Fakultas Ilmu Komputer Universitas Jember memiliki akreditasi A dari BAN-PT.
* Program studi ini memiliki visi untuk menghasilkan lulusan yang kompeten dalam bidang sistem informasi dan dapat mengembangkan solusi teknologi informasi yang inovatif.
* Kurikulum program studi Sistem Informasi di Fakultas Ilmu Komputer Universitas Jember mencakup mata kuliah seperti Pengantar Sistem Informasi, Basis Data, Pemrograman, Analisis dan Desain Sistem, dan lain-lain.

Untuk mendaftar ke program studi Sistem Informasi di Fakultas Ilmu Komputer Universitas Jember, Anda dapat mengikuti langkah-langkah berikut:

* Pastikan Anda telah memenuhi persyaratan pendaftaran, seperti memiliki ijazah SMA/SMK/MA yang relevan dan memiliki nilai yang memenuhi syarat.
* Pilih jalur pendaftaran yang sesuai, seperti SNMPTN, SBMPTN, atau Mandiri.
* Isi formulir pendaftaran online dan unggah dokumen yang diperlukan.
* Ikuti tes seleksi masuk, jika diperlukan.
* Tunggu pengumuman hasil seleksi masuk.

Anda dapat memeriksa situs web resmi Universitas Jember untuk mendapatkan informasi lebih lanjut tentang pendaftaran dan program studi Sistem Informasi di Fakultas Ilmu Komputer.

Terima kasih!

In [20]:
answer = get_response(
    query="Apakah ada beasiswa yang bisa saya dapatkan di universitas ini?",
    context=context,
    )

display(Markdown(answer))

Halo! Ya, Universitas Jember menawarkan beberapa jenis beasiswa untuk mahasiswanya. Informasi lebih lanjut tentang beasiswa dapat ditemukan di website resmi Universitas Jember, khususnya di halaman Kemahasiswaan. 
Terima kasih!

In [22]:
answer = get_response(
    query="Apa saja program kemahasiswaa yang ada di universitas ini?",
    context=context,
    )

display(Markdown(answer))

Halo! Terima kasih telah bertanya tentang program kemahasiswaan di Universitas Jember. 

Berdasarkan informasi yang tersedia di website resmi Universitas Jember, beberapa program kemahasiswaan yang ada di universitas ini antara lain:

1. Program Merdeka Belajar Kampus Merdeka (MBKM)
2. Asistensi Mengajar
3. Kegiatan Mahasiswa
4. Program Beasiswa

Untuk informasi lebih lanjut, silakan mengunjungi website resmi Universitas Jember atau menghubungi Kemahasiswaan Universitas Jember.

Terima kasih!

## With User Interface from Gradio

In [29]:
def chat_interface(query, context=""):
    if not query:
        return "Maaf, pertanyaan tidak boleh kosong."
    else:
        try:
            response = get_response(query, context)
            return response
        except Exception as e:
            return f"Error: {str(e)}"

siubot = gr.Interface(
    fn=chat_interface,
    inputs=[
        gr.Textbox(label="Pertanyaan", placeholder="Ketik pertanyaanmu di sini..."),
        gr.Textbox(label="Konteks (Opsional)", placeholder="Berikan konteks pertanyaanmu di sini (opsional)..."),
    ],
    outputs=gr.Textbox(label="Asisten Kemahasiswaan"),
    title="SIUBOT - UNEJ Chatbot",
    description="Selamat datang di SIUBOT! Silakan tanyakan pertanyaan apapun seputar Universitas Jember.",
)

siubot.launch(pwa=True, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eff05217d4a792970e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## **Limitations**  
This simple chatbot has several limitations, including:  
1. The chatbot only provides answers based on the information available on the official University of Jember website.  
2. It can only respond to questions in **Indonesian**.  

## **Development Suggestions**  
Future improvements may include using a more advanced language model or chatbot architecture to enhance response accuracy. Implementing **Retrieval-Augmented Generation (RAG)** could improve inference by incorporating internal data for more relevant answers. Additionally, expanding multilingual support would allow the chatbot to respond in multiple languages based on user preferences. Further enhancements could also include a more comprehensive **user interface** for a better user experience.

The **SIUBOT** model is currently under development to become more **flexible and adaptive**. Stay tuned for updates! 🚀